In [2]:
# pipenv install pandas plotly scikit-learn lightgbm bayesian-optimization ipykernel ipywidgets nbformat matplotlib
# No Mac, instalar biblioteca libomp: brew install libomp

# Importar bibliotecas
import pandas as pd
import numpy as np

# Visualização
import plotly.express as px
import matplotlib.pyplot as plt

# Machine Learning
from lightgbm import LGBMClassifier, early_stopping, plot_tree, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, \
                            ConfusionMatrixDisplay, log_loss

# Otimização Bayesiana
from bayes_opt import BayesianOptimization

### Carregar os dados

In [3]:
df_vinhos = pd.read_csv('./datasets/wine_data.csv')

### Visualizar Estrutura

In [12]:
df_vinhos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         21000 non-null  float64
 1   volatile_acidity      21000 non-null  float64
 2   citric_acid           21000 non-null  float64
 3   residual_sugar        21000 non-null  float64
 4   chlorides             21000 non-null  float64
 5   free_sulfur_dioxide   21000 non-null  float64
 6   total_sulfur_dioxide  21000 non-null  float64
 7   density               21000 non-null  float64
 8   pH                    21000 non-null  float64
 9   sulphates             21000 non-null  float64
 10  alcohol               21000 non-null  float64
 11  quality               21000 non-null  int64  
dtypes: float64(11), int64(1)
memory usage: 1.9 MB


In [13]:
# Visualizar primeiros registros
df_vinhos.head(10)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,11.6,0.580,0.66,2.20,0.074,10.0,47.0,1.00080,3.25,0.57,9.00,3
1,10.4,0.610,0.49,2.10,0.200,5.0,16.0,0.99940,3.16,0.63,8.40,3
2,7.4,1.185,0.00,4.25,0.097,5.0,14.0,0.99660,3.63,0.54,10.70,3
3,10.4,0.440,0.42,1.50,0.145,34.0,48.0,0.99832,3.38,0.86,9.90,3
4,8.3,1.020,0.02,3.40,0.084,6.0,11.0,0.99892,3.48,0.49,11.00,3
5,7.6,1.580,0.00,2.10,0.137,5.0,9.0,0.99476,3.50,0.40,10.90,3
6,6.8,0.815,0.00,1.20,0.267,16.0,29.0,0.99471,3.32,0.51,9.80,3
7,7.3,0.980,0.05,2.10,0.061,20.0,49.0,0.99705,3.31,0.55,9.70,3
8,7.1,0.875,0.05,5.70,0.082,3.0,14.0,0.99808,3.40,0.52,10.20,3
9,6.7,0.760,0.02,1.80,0.078,6.0,12.0,0.99600,3.55,0.63,9.95,3


In [14]:
# Visualizar últimos registros
df_vinhos.tail(10)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
20990,12.3,1.21,0.68,59.1,0.328,232.0,396.0,1.03428,3.12,0.59,12.2,9
20991,10.2,0.61,0.88,53.8,0.250,62.4,204.7,1.02776,3.52,1.14,9.7,9
20992,12.9,0.90,1.16,51.2,0.309,196.9,172.3,1.03000,3.10,0.82,12.4,9
20993,13.0,0.58,1.22,52.8,0.247,93.0,190.7,1.01922,3.38,1.24,9.9,9
20994,12.8,0.85,1.12,44.9,0.188,215.3,253.3,1.02192,3.13,0.81,11.4,9
20995,9.7,1.02,0.91,50.0,0.412,114.6,181.7,1.02085,3.30,0.89,12.0,9
20996,10.2,0.61,0.88,53.8,0.250,62.4,204.7,1.02776,3.52,1.14,9.7,9
20997,13.4,0.46,1.04,52.1,0.449,63.0,273.5,1.02618,2.89,1.76,9.3,9
20998,6.6,1.03,1.09,25.3,0.138,179.8,295.0,1.02476,2.94,1.54,12.9,9
20999,9.3,0.93,1.32,33.6,0.412,128.7,290.1,1.02182,3.16,1.42,13.0,9


In [15]:
# Estatisitcas descritivas
df_vinhos.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
count,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000
mean,9.797079,0.774796,0.793870,31.289348,0.200245,129.442333,229.008762,1.009972,3.158712,1.020641,11.291716,6.000000
std,2.413919,0.365015,0.384833,19.015391,0.124933,77.167262,100.183265,0.012032,0.171371,0.408304,1.182198,2.000048
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,7.600000,0.430000,0.410000,9.800000,0.072000,45.000000,150.000000,0.997417,3.030000,0.620000,10.400000,4.000000
50%,10.000000,0.830000,0.870000,37.600000,0.205000,145.800000,240.500000,1.012200,3.150000,1.080000,11.300000,6.000000
75%,11.800000,1.080000,1.110000,46.800000,0.298000,194.325000,311.625000,1.019840,3.270000,1.360000,12.200000,8.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000
